# 基础知识库（Bedrock Knowledge Bases）

[Amazon Bedrock的知识库](https://aws.amazon.com/bedrock/knowledge-bases/)是亚马逊网络服务（AWS）的一项服务，它允许您通过使用私有数据来定制FM响应，快速构建RAG应用程序。

实现`RAG`需要组织执行多个繁琐的步骤，将数据转换为嵌入（向量），将嵌入存储在专门的向量数据库中，并构建定制集成到数据库中，以搜索和检索与用户查询相关的文本。这可能是耗时且低效的。

使用`Amazon Bedrock的知识库`，只需指向您在`Amazon S3`中数据的位置，`Amazon Bedrock的知识库`会处理整个摄入工作流程到您的向量数据库中。如果您没有现有的向量数据库，Amazon Bedrock会为您创建一个Amazon OpenSearch无服务器向量存储。

可以通过[AWS控制台](https://aws.amazon.com/console/)或使用[AWS SDKs](https://aws.amazon.com/developer/tools/)来配置知识库。

在这个笔记本中，我们介绍了AmazonKnowledgeBasesRetriever - Amazon Bedrock在Llama Index中的集成，通过检索API从知识库中检索与用户查询相关的结果。


## 使用知识库检索器


In [ ]:
%pip install --upgrade --quiet  boto3 botocore
%pip install llama-index
%pip install llama-index-retrievers-bedrock

有关`retrieval_config`支持的参数的更多信息，请查看boto3文档：[链接](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)

要在`retrieval_config`中使用过滤器，您需要为数据源设置metadata.json文件。有关更多信息，请参见：[链接](https://aws.amazon.com/blogs/machine-learning/knowledge-bases-for-amazon-bedrock-now-supports-metadata-filtering-to-improve-retrieval-accuracy/)


In [ ]:
from llama_index.retrievers.bedrock import AmazonKnowledgeBasesRetriever

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="<knowledge-base-id>",
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
            "overrideSearchType": "HYBRID",
            "filter": {"equals": {"key": "tag", "value": "space"}},
        }
    },
)

In [ ]:
查询 = "和整个宇宙相比，银河系有多大？"检索结果 = 检索器.检索(查询)# 打印第一个检索结果打印(检索结果[0].获取内容())

## 使用检索器查询Bedrock LLMs

```python
from transformers import pipeline

retriever = pipeline("retrieval", model="facebook/rag-token-nq")
retriever("What is the capital of France?")
```


In [ ]:
%pip install llama-index-llms-bedrock

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.llms.bedrock.base import Bedrock

llm = Bedrock(model="anthropic.claude-v2", temperature=0, max_tokens=3000)
response_synthesizer = get_response_synthesizer(
    response_mode="compact", llm=llm
)
response_obj = response_synthesizer.synthesize(query, retrieved_results)
print(response_obj)